# Data Import

In [1]:
import vaex

ERROR:MainThread:vaex:issue loading reader
Traceback (most recent call last):
  File "/home/alex/miniconda3/envs/test_env/lib/python3.7/site-packages/vaex/__init__.py", line 720, in <module>
    add_namespace = entry.load()
  File "/home/alex/miniconda3/envs/test_env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 2449, in load
    self.require(*args, **kwargs)
  File "/home/alex/miniconda3/envs/test_env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 2472, in require
    items = working_set.resolve(reqs, env, installer, extras=self.extras)
  File "/home/alex/miniconda3/envs/test_env/lib/python3.7/site-packages/pkg_resources/__init__.py", line 792, in resolve
    raise VersionConflict(dist, req).with_context(dependent_req)
pkg_resources.VersionConflict: (pyarrow 0.12.1 (/home/alex/miniconda3/envs/test_env/lib/python3.7/site-packages), Requirement.parse('pyarrow>=0.15'))


In [2]:
path = "../data/portals.csv"

In [3]:
data = vaex.from_csv(path)

In [4]:
data

#,_id,address,description,discoverers,late6,lnge6,location,name,player,timestamp,updated,usage,index
0,ObjectId(5c221b59816b243163f1465a),"85 Neil Road, Singapore 088845",nan,"[{""agent"":""FuzzyPillow"",""team"":""RESISTANCE""}]",1279127,103842105,"'{""type"":""Point"",""coordinates"":[103.842105,1.279...",Singapore Institute of Architects,_NEUTRAL_,2020-07-19T05:35:34.367Z,2020-07-19T05:35:34.367Z,"'{""timestamp"":{""$numberLong"":""1592817612118""},""p...",0
1,ObjectId(5c221b59816b243163f1465b),"'Jalan Cigadung Wetan, Cibeunying Kaler, Bandung...",nan,"[{""agent"":""Shiyoga"",""team"":""ALIENS""}]",-6882850,107629399,"'{""type"":""Point"",""coordinates"":[107.629399,-6.88...",Stand Alone Statue,AngelllDemons,2020-05-19T02:21:54.000Z,2020-07-17T17:38:05.937Z,"'{""timestamp"":{""$numberLong"":""1590906934191""},""p...",1
2,ObjectId(5c221b59816b243163f1465c),"'Swiss Hotel, George Town, 10450 George Town, Pu...",Mr. Bean cycling at Chulia St.,"[{""agent"":""UEFISOCFPGA"",""team"":""RESISTANCE""}]",5418961,100334498,"'{""type"":""Point"",""coordinates"":[100.334498,5.418...",Mr. Bean cycling at Chulia St.,_NEUTRAL_,2020-07-19T17:43:17.986Z,2020-07-19T17:43:17.986Z,"'{""timestamp"":{""$numberLong"":""1595154205516""},""p...",2
3,ObjectId(5c221b59816b243163f1465d),"'392, Lebuh Chulia, George Town, 10200 George To...",Tiger mural,"[{""agent"":""ernie1601"",""team"":""ALIENS""}]",5419246,100334373,"'{""type"":""Point"",""coordinates"":[100.334373,5.419...",Tiger Mural,ENlM4RCU5,2020-07-19T10:23:25.000Z,2020-07-19T10:23:25.000Z,"'{""timestamp"":{""$numberLong"":""1595154205224""},""p...",3
4,ObjectId(5c221b59816b243163f1465e),"932 Hougang Ave 9, Singapore 532932",nan,"[{""agent"":""ReZ3L"",""team"":""RESISTANCE""}]",1375444,103882176,"'{""type"":""Point"",""coordinates"":[103.882176,1.375...",932 Playground,JansonSG,2020-02-15T18:05:26.000Z,2020-07-19T01:45:04.441Z,"'{""timestamp"":{""$numberLong"":""1592866236560""},""p...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"12,506,624",ObjectId(5f151a1f374cbf4c844a2421),"'Avenida Mérida 2000, Yucalpetén, Hacienda Mulsa...",nan,nan,20986401,-89663431,"'{""type"":""Point"",""coordinates"":[-89.663431,20.98...",Templo “CCIRS”,_NEUTRAL_,2020-07-20T04:14:23.168Z,2020-07-20T04:14:23.168Z,nan,12506624
"12,506,625",ObjectId(5f151acf8778710001bcc0a1),"'227 Gumi-dong, Bundang-gu, Seongnam-si, Gyeongg...",nan,[],37340149,127122628,"'{""type"":""Point"",""coordinates"":[127.122628,37.34...",구미동 행정복지센터,HeeThee,2020-06-21T09:15:00.000Z,2020-07-20T04:18:17.979Z,"'{""timestamp"":{""$numberLong"":""1592730889310""},""p...",12506625
"12,506,626",ObjectId(5f151ae2374cbf4c844afa78),"Kimmstraße 7, 28757 Bremen, Germany",nan,nan,53172632,8617439,nan,Das Steuerrad,Stoody,2020-07-20T04:16:59.000Z,2020-07-20T04:16:59.000Z,"'{""timestamp"":{""$numberLong"":""1595218670806""},""p...",12506626
"12,506,627",ObjectId(5f151b3e8778710001bcc0a2),"Guzman Street, Davao City, Philippines",nan,"'[{""agent"":""iILunggoy"",""team"":""ALIENS""},{""agent""...",7078299,125615119,"'{""type"":""Point"",""coordinates"":[125.615119,7.078...",Mary Sculpture Fountain,_NEUTRAL_,2020-07-20T04:19:29.249Z,2020-07-20T04:19:29.249Z,"'{""timestamp"":{""$numberLong"":""1591947394849""},""p...",12506627


In [5]:
sample_1k = data.sample(n=1000)
sample_10k = data.sample(n=10000)
sample_100k = data.sample(n=100000)

In [31]:
sample_1k.to_pandas_df().to_csv("../data/portals_1k.csv")
sample_10k.to_pandas_df().to_csv("../data/portals_10k.csv")
sample_100k.to_pandas_df().to_csv("../data/portals_100k.csv")

# Reverse Geocoding

In [5]:
import json
import time

import numpy as np
import pandas as pd

from IPython.display import clear_output

from geopy.point import Point
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [6]:
samp = pd.read_csv("../data/portals_1k.csv")
geolocator = Nominatim(user_agent="alex.melkoff@gmail.com")
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def empty_rec():
    return {
        "village": None,
        "town": None,
        "city": None,
        "country": None,
        "postcode": None,
        "country_code": None,
    }

def geocode_wrapper(row):
    clear_output()
    nomi_data = None
    print("processing", row.address)
    
    if row.address:
        nomi_data = geolocator.geocode(row.address, addressdetails=True, exactly_one=True, language="en")

    if not nomi_data:
        print("fallback to reverse by location...")
        pt = Point(list(reversed(json.loads(row.location)["coordinates"])))
        nomi_data = geolocator.reverse(pt, addressdetails=True, exactly_one=True, language="en")

    if not "address" in nomi_data.raw:
        print("skipping: no address...")
        return pd.Series(empty_rec())

    nomi_address = nomi_data.raw["address"]
    empt = empty_rec()
    nomi_address = {k: v for k, v in nomi_address.items() if k in empt}
    empt.update(nomi_address)
    time.sleep(1)
    return pd.Series(empt)

nomi_data = samp.apply(lambda x: geocode_wrapper(x), axis=1)
samp = samp.join(nomi_data, how="left")
samp.to_csv("../data/portals_1k_addr.csv")
samp

processing 1717-1 Shimonumabe, Nakahara-ku, Kawasaki, Kanagawa 211-0011, Japan
fallback to reverse by location...


,Unnamed: 0,_id,address,description,discoverers,late6,lnge6,location,name,player,timestamp,updated,usage,index,village,town,city,country,postcode,country_code
0,0,ObjectId(5c223622816b2431634e8b19),"Hylperdijk 2, 8711 HJ Workum, Netherlands",NaN,"[{""agent"":""ellende"",""team"":""ALIENS""}]",52967186,5426965,"{""type"":""Point"",""coordinates"":[5.426965,52.967...",Molen Workum,_NEUTRAL_,2020-07-18T13:52:24.620Z,2020-07-18T13:52:24.620Z,"{""timestamp"":{""$numberLong"":""1588103535528""},""...",5366552,None,None,Súdwest-Fryslân,The Netherlands,8711HK,nl
1,1,ObjectId(5c65115063c50c1d51e70e9a),"Stienitzseestraße 33, 12489 Berlin, Germany",NaN,"[{""agent"":""Si9"",""team"":""ALIENS""}]",52440244,13549760,"{""type"":""Point"",""coordinates"":[13.54976,52.440...",Erich Steinfurth,NaN,2012-11-15T00:00:00.000Z,2019-02-14T06:57:20.000Z,"{""timestamp"":{""$numberLong"":""1490000000000""},""...",7928040,None,None,Berlin,Germany,12489,de
2,2,ObjectId(5dd0ee0dbf5db3472afc744c),"Rua Edgar Torres, 878 - Minas Caixa, Belo Hori...",Arte urbana,[],-19807552,-43954239,"{""type"":""Point"",""coordinates"":[-43.954239,-19....",Grafite Irmãos Metralha,_NEUTRAL_,2020-07-19T15:15:07.555Z,2020-07-19T15:15:07.555Z,"{""timestamp"":{""$numberLong"":""1580516993627""},""...",9785072,None,None,Belo Horizonte,Brazil,31615250,br
3,3,ObjectId(5e9d2a5e402e17f771e8e238),"U 5 ježků, Podzásepská, Hoděšovice, Býšť, okre...",NaN,[],50168534,15926050,"{""type"":""Point"",""coordinates"":[15.92605,50.168...",Dendrofon,_NEUTRAL_,2020-07-18T09:28:38.897Z,2020-07-18T09:28:38.897Z,NaN,11496402,Býšť,None,None,Czech Republic,533 22,cz
4,4,ObjectId(5c22358a816b24316329ac0d),"Zámok 2267/1, 920 01 Hlohovec, Slovakia",NaN,"[{""agent"":""doktorlevin"",""team"":""RESISTANCE""},{...",48419408,17788097,"{""type"":""Point"",""coordinates"":[17.788097,48.41...",Tabula Polepsovne,_NEUTRAL_,2020-07-19T01:43:30.537Z,2020-07-19T01:43:30.537Z,"{""timestamp"":{""$numberLong"":""1574613857921""},""...",3026947,None,None,District of Hlohovec,Slovakia,920 01,sk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,ObjectId(5c2234f6816b243163fd992c),"LOT 295 Kulin-Lake Grace Road, Kulin WA 6365, ...",NaN,"[{""agent"":""seefit"",""team"":""ALIENS""}]",-32668276,118159377,"{""type"":""Point"",""coordinates"":[118.159377,-32....",Tin Horse,_NEUTRAL_,2020-07-19T22:27:48.560Z,2020-07-19T22:27:48.560Z,"{""timestamp"":{""$numberLong"":""1575684753183""},""...",220775,None,Kulin,None,Australia,None,au
996,996,ObjectId(5c2235ae816b243163328fdd),"219 Thomas Street, Hampton VIC 3188, Australia",Sports centre,"[{""agent"":""Ezio4uditore"",""team"":""RESISTANCE""}]",-37939974,145017804,"{""type"":""Point"",""coordinates"":[145.017804,-37....",Jack Brockhoff Sports Centre,_NEUTRAL_,2020-07-19T23:50:20.474Z,2020-07-19T23:50:20.474Z,"{""timestamp"":{""$numberLong"":""1591668828343""},""...",3589304,None,None,None,Australia,3188,au
997,997,ObjectId(5de091ee1d69b2b9bea92ec3),"Voorstraat 75, 8801 LB Franeker, Netherlands","Jan Willem de Crane, professor geschiedenis en...",[],53186501,5539526,"{""type"":""Point"",""coordinates"":[5.539526,53.186...",Hier woonde Jan Willem de Crane,_NEUTRAL_,2020-07-18T02:19:10.295Z,2020-07-18T02:19:10.295Z,"{""timestamp"":{""$numberLong"":""1590766246612""},""...",10066703,None,None,Waadhoeke,The Netherlands,8801LB,nl
998,998,ObjectId(5c223579816b24316325396a),"6902 Tuxedo Road, San Diego, CA 92119, USA",NaN,"[{""agent"":""alphuris"",""team"":""ALIENS""},{""agent""...",32809423,-117048005,"{""type"":""Point"",""coordinates"":[-117.048005,32....",Tuxedo Neighborhood Park,BoConstrictor,2020-06-29T21:14:15.000Z,2020-07-19T23:32:10.615Z,"{""timestamp"":{""$numberLong"":""1594068804740""},""...",2743066,None,None,San Diego,United States of America,92119,us


# Formatting

In [7]:
portals = pd.read_csv("../data/portals_1k_addr.csv", parse_dates=["timestamp", "updated"])

def concat_city_town(row):
    ct = row.city
    if ct is np.NaN:
        ct = row.town
    if ct is np.NaN:
        ct = row.village
    return ct

def unwrap_location(row):
    lon, lat = json.loads(row.location)["coordinates"]
    return pd.Series({"lat": lat, "lon": lon})


def unwrap_discoverers(row):
    discoverers = json.loads(row.discoverers)
    if not discoverers:
        return pd.Series({"team": None, "agent": None, "team_size": 0})
    return pd.Series(
        {
            "team": discoverers[0]["team"],
            "agent": discoverers[0]["agent"],
            "team_size": len(discoverers),
        }
    )

portals = portals.join(pd.Series(portals.apply(concat_city_town, axis=1), name="ct"), how="left")
portals = portals.join(portals.apply(unwrap_location, axis=1), how="left")
portals = portals.join(portals.apply(unwrap_discoverers, axis=1), how="left")

portals["team"] = portals.team.fillna("MISSING")
portals["country_code"] = portals.country_code.apply(lambda x: str(x).upper())

portals = portals.drop(["discoverers", "location"], axis=1)
portals.to_csv("../data/portals_1k_addr_formatted.csv")
portals

,Unnamed: 0,Unnamed: 0.1,_id,address,description,late6,lnge6,name,player,timestamp,...,city,country,postcode,country_code,ct,lat,lon,team,agent,team_size
0,0,0,ObjectId(5c223622816b2431634e8b19),"Hylperdijk 2, 8711 HJ Workum, Netherlands",NaN,52967186,5426965,Molen Workum,_NEUTRAL_,2020-07-18 13:52:24.620000+00:00,...,Súdwest-Fryslân,The Netherlands,8711HK,NL,Súdwest-Fryslân,52.967186,5.426965,ALIENS,ellende,1.0
1,1,1,ObjectId(5c65115063c50c1d51e70e9a),"Stienitzseestraße 33, 12489 Berlin, Germany",NaN,52440244,13549760,Erich Steinfurth,NaN,2012-11-15 00:00:00+00:00,...,Berlin,Germany,12489,DE,Berlin,52.440244,13.549760,ALIENS,Si9,1.0
2,2,2,ObjectId(5dd0ee0dbf5db3472afc744c),"Rua Edgar Torres, 878 - Minas Caixa, Belo Hori...",Arte urbana,-19807552,-43954239,Grafite Irmãos Metralha,_NEUTRAL_,2020-07-19 15:15:07.555000+00:00,...,Belo Horizonte,Brazil,31615250,BR,Belo Horizonte,-19.807552,-43.954239,MISSING,NaN,0.0
3,3,3,ObjectId(5e9d2a5e402e17f771e8e238),"U 5 ježků, Podzásepská, Hoděšovice, Býšť, okre...",NaN,50168534,15926050,Dendrofon,_NEUTRAL_,2020-07-18 09:28:38.897000+00:00,...,NaN,Czech Republic,533 22,CZ,Býšť,50.168534,15.926050,MISSING,NaN,0.0
4,4,4,ObjectId(5c22358a816b24316329ac0d),"Zámok 2267/1, 920 01 Hlohovec, Slovakia",NaN,48419408,17788097,Tabula Polepsovne,_NEUTRAL_,2020-07-19 01:43:30.537000+00:00,...,District of Hlohovec,Slovakia,920 01,SK,District of Hlohovec,48.419408,17.788097,RESISTANCE,doktorlevin,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,ObjectId(5c2234f6816b243163fd992c),"LOT 295 Kulin-Lake Grace Road, Kulin WA 6365, ...",NaN,-32668276,118159377,Tin Horse,_NEUTRAL_,2020-07-19 22:27:48.560000+00:00,...,NaN,Australia,NaN,AU,Kulin,-32.668276,118.159377,ALIENS,seefit,1.0
996,996,996,ObjectId(5c2235ae816b243163328fdd),"219 Thomas Street, Hampton VIC 3188, Australia",Sports centre,-37939974,145017804,Jack Brockhoff Sports Centre,_NEUTRAL_,2020-07-19 23:50:20.474000+00:00,...,NaN,Australia,3188,AU,NaN,-37.939974,145.017804,RESISTANCE,Ezio4uditore,1.0
997,997,997,ObjectId(5de091ee1d69b2b9bea92ec3),"Voorstraat 75, 8801 LB Franeker, Netherlands","Jan Willem de Crane, professor geschiedenis en...",53186501,5539526,Hier woonde Jan Willem de Crane,_NEUTRAL_,2020-07-18 02:19:10.295000+00:00,...,Waadhoeke,The Netherlands,8801LB,NL,Waadhoeke,53.186501,5.539526,MISSING,NaN,0.0
998,998,998,ObjectId(5c223579816b24316325396a),"6902 Tuxedo Road, San Diego, CA 92119, USA",NaN,32809423,-117048005,Tuxedo Neighborhood Park,BoConstrictor,2020-06-29 21:14:15+00:00,...,San Diego,United States of America,92119,US,San Diego,32.809423,-117.048005,ALIENS,alphuris,5.0
